<a href="https://colab.research.google.com/github/ymsang/AI_projects/blob/main/Copy_of_n222a_random_forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 2 / NOTE 2*

# 📝 Assignment
---

# 랜덤포레스트(Random Forests)

### 1) 캐글 대회를 이어서 진행합니다. EDA, 데이터 전처리 부분을 업데이트 하세요.
- EDA는 항상 완벽할 수 없지요 한 주간 계속 같은 데이터로 과제를 진행하므로 부족한 부분을 추가하거나 논하세요.
- (지금은 feature engineering에 너무 시간을 들이지 마세요!)
- Ordinal Encoding을 적용해 보세요.
- **(객관식) 다음 특성들 중에 순서를 고려한 Ordinal Encoding이 필요해 보이는 특성을 선택하고 과제 제출폼에 제출하세요.**
    1. opinion_h1n1_vacc_effective
    2. state
    3. marital
    4. employment_occupation


In [16]:
### 이곳에서 과제를 진행해 주세요 ###

import pandas as pd

target = 'vacc_h1n1_f'
# target = 'vacc_seas_f'
train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [17]:
train.head().T

,0,1,2,3,4
h1n1_concern,NaN,3,2,2,1
h1n1_knowledge,NaN,1,1,2,2
behavioral_antiviral_meds,NaN,0,0,1,0
behavioral_avoidance,NaN,0,1,1,1
behavioral_face_mask,NaN,0,0,0,0
behavioral_wash_hands,NaN,1,1,1,1
behavioral_large_gatherings,NaN,0,1,0,0
behavioral_outside_home,NaN,0,0,0,0
behavioral_touch_face,NaN,1,1,0,1
doctor_recc_h1n1,0,0,0,0,0


In [18]:
#train 을 훈련/검증 세트로 분리

from sklearn.model_selection import train_test_split
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train[target], random_state=2)


train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

In [5]:
!pip install category_encoders

     |████████████████████████████████| 81kB 3.2MB/s 


In [20]:
#ordinal encoder

from category_encoders import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(train)


OrdinalEncoder(cols=['opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
                     'opinion_h1n1_sick_from_vacc',
                     'opinion_seas_vacc_effective', 'opinion_seas_risk',
                     'opinion_seas_sick_from_vacc', 'agegrp',
                     'employment_status', 'census_msa', 'employment_industry',
                     'employment_occupation', 'state'],
               drop_invariant=False, handle_missing='value',
               handle_unknown='value',
               mappi...
ILLINOIS               22
RHODE ISLAND           23
VIRGINIA               24
COLORADO               25
SOUTH DAKOTA           26
MICHIGAN               27
VERMONT                28
KANSAS                 29
INDIANA                30
ARIZONA                31
WYOMING                32
NEW YORK               33
NORTH CAROLINA         34
CALIFORNIA             35
MISSOURI               36
NEW MEXICO             37
OHIO                   38
TENNESSEE              39
FLORIDA      

### 2) 랜덤포레스트 모델을 적용한 후의 결과를 캐글에 제출하세요.
- 랜덤포레스트를 적용하고 성능이 오히려 떨어졌을 수도 있습니다! 만약 그렇다면 이유를 본인 논리로 분석해 보세요.
- **캐글 Leaderboard의 본인 Score를 과제 제출폼에 제출하세요.**

In [21]:
### 이곳에서 과제를 진행해 주세요 ###

def engineer(df):
    """특성을 엔지니어링 하는 함수입니다."""
    
    # 높은 카디널리티를 가지는 특성을 제거합니다.
#     selected_cols = df.select_dtypes(include=['number', 'object'])
#     colnames = selected_cols.columns.tolist()
#     labels = selected_cols.nunique()
    
#     selected_features = labels[labels <= 30].index.tolist()
#     df = df[selected_features]
    
    # 새로운 특성을 생성합니다.
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
        
    return df


train = engineer(train)
val = engineer(val)
test = engineer(test)

In [22]:
# 훈련/검증/테스트 데이터를 특성과 타겟으로 분리

features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [44]:
#랜덤포레스트 적용

from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

# ordinal encoding
pipe_ord = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=10, n_jobs=-1, oob_score=True)
)

pipe_ord.fit(X_train, y_train)
print('검증 정확도', pipe_ord.score(X_val, y_val))



검증 정확도 0.8254062388803226


In [46]:
y_test_pred = pipe.predict(X_test)
y_test_pred

array([0, 0, 0, ..., 0, 1, 0])

submission 파일 만들기

In [24]:
sample_submission

,Id,vacc_h1n1_f
0,0,0
1,1,1
2,2,1
3,3,1
4,4,0
...,...,...
28099,28099,0
28100,28100,1
28101,28101,1
28102,28102,0


In [47]:
submission = sample_submission.copy()
# display(submission.head())
submission['vacc_h1n1_f'] = y_test_pred
submission = submission.set_index('Id')
# display(submission.head())
submission.to_csv('submission_v2_1.csv')

## 🔥 도전과제

### 3) 수업에 사용하지 않은 다른 종류의 [category_encoders](http://contrib.scikit-learn.org/category_encoders/)을 적용해 보고 인코더에 대해 설명하세요


In [ ]:
### 이곳에서 과제를 진행해 주세요 ###

## 참고자료

- [Random Forests for Complete Beginners](https://victorzhou.com/blog/intro-to-random-forests/)